In [1]:
# from sklearn.ensemble import VotingClassifier
# log_clf = LogisticRegression() rnd_clf = RandomForestClassifier() svm_clf = SVC()
# voting_clf = VotingClassifier(        
#     estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
#     voting='soft'
# ) 
# voting_clf.fit(X_train, y_train) 
# from sklearn.metrics import accuracy_score
# for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
#     print(clf.__class__.__name__, accuracy_score(y_test, y_pred)) 


In [2]:
#from google.colab import drive
#drive.mount("/content/drive")


import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, KFold
#pd.set_option('display.max_columns', None)


In [3]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sample = pd.read_csv('SampleSubmission.csv')

In [4]:
#Exploratory Data Analysis

import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


print("train set shape: ", train.shape)
print("test set shape: ", test.shape)


train set shape:  (56000, 52)
test set shape:  (24000, 51)


In [5]:
train.info()

test.info()

train.head()

test.head()

train.isnull().sum()

test.isnull().sum()

train.fillna(-99999, inplace=True)
test.fillna(-99999, inplace=True)

train.isnull().sum()

test.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 52 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Applicant_ID    56000 non-null  object 
 1   form_field1     53471 non-null  float64
 2   form_field2     52156 non-null  float64
 3   form_field3     55645 non-null  float64
 4   form_field4     55645 non-null  float64
 5   form_field5     55645 non-null  float64
 6   form_field6     42640 non-null  float64
 7   form_field7     50837 non-null  float64
 8   form_field8     42640 non-null  float64
 9   form_field9     47992 non-null  float64
 10  form_field10    55645 non-null  float64
 11  form_field11    24579 non-null  float64
 12  form_field12    46105 non-null  float64
 13  form_field13    50111 non-null  float64
 14  form_field14    56000 non-null  int64  
 15  form_field15    33525 non-null  float64
 16  form_field16    42964 non-null  float64
 17  form_field17    44849 non-null 

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,...,NaN,0.825000,1.01,0.800000,NaN,0.0,charge,NaN,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,...,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,NaN,0.504974,0.043525
2,Apcnt_1000052,NaN,0.27505,0.0600,0.0000,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.00,NaN,NaN,NaN,charge,NaN,0.000000,NaN
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,...,NaN,0.916663,2.02,0.464224,NaN,NaN,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,...,NaN,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124


Applicant_ID          0
form_field1        2529
form_field2        3844
form_field3         355
form_field4         355
form_field5         355
form_field6       13360
form_field7        5163
form_field8       13360
form_field9        8008
form_field10        355
form_field11      31421
form_field12       9895
form_field13       5889
form_field14          0
form_field15      22475
form_field16      13036
form_field17      11151
form_field18      10402
form_field19          4
form_field20        355
form_field21      15854
form_field22      20400
form_field23      28123
form_field24      13297
form_field25       5450
form_field26       7438
form_field27       9299
form_field28        355
form_field29        355
form_field30      25509
form_field31      39408
form_field32       5450
form_field33       1256
form_field34        355
form_field35      23148
form_field36       1995
form_field37       5450
form_field38        355
form_field39       4211
form_field40      43729
form_field41    

Applicant_ID        0
form_field1      1110
form_field2      1709
form_field3       146
form_field4       146
form_field5       146
form_field6      5604
form_field7      2231
form_field8      5604
form_field9      3400
form_field10      147
form_field11    13398
form_field12     4183
form_field13     2463
form_field14        0
form_field15     9592
form_field16     5474
form_field17     4695
form_field18     4369
form_field19        0
form_field20      147
form_field21     6707
form_field22     8724
form_field23    12125
form_field24     5605
form_field25     2256
form_field26     3172
form_field27     3910
form_field28      147
form_field29      147
form_field30    10908
form_field31    16810
form_field32     2256
form_field33      495
form_field34      147
form_field35     9866
form_field36      903
form_field37     2256
form_field38      147
form_field39     1829
form_field40    18828
form_field41    16349
form_field42      578
form_field43      250
form_field44     2362
form_field

Applicant_ID      0
form_field1       0
form_field2       0
form_field3       0
form_field4       0
form_field5       0
form_field6       0
form_field7       0
form_field8       0
form_field9       0
form_field10      0
form_field11      0
form_field12      0
form_field13      0
form_field14      0
form_field15      0
form_field16      0
form_field17      0
form_field18      0
form_field19      0
form_field20      0
form_field21      0
form_field22      0
form_field23      0
form_field24      0
form_field25      0
form_field26      0
form_field27      0
form_field28      0
form_field29      0
form_field30      0
form_field31      0
form_field32      0
form_field33      0
form_field34      0
form_field35      0
form_field36      0
form_field37      0
form_field38      0
form_field39      0
form_field40      0
form_field41      0
form_field42      0
form_field43      0
form_field44      0
form_field45      0
form_field46      0
form_field47      0
form_field48      0
form_field49      0


Applicant_ID    0
form_field1     0
form_field2     0
form_field3     0
form_field4     0
form_field5     0
form_field6     0
form_field7     0
form_field8     0
form_field9     0
form_field10    0
form_field11    0
form_field12    0
form_field13    0
form_field14    0
form_field15    0
form_field16    0
form_field17    0
form_field18    0
form_field19    0
form_field20    0
form_field21    0
form_field22    0
form_field23    0
form_field24    0
form_field25    0
form_field26    0
form_field27    0
form_field28    0
form_field29    0
form_field30    0
form_field31    0
form_field32    0
form_field33    0
form_field34    0
form_field35    0
form_field36    0
form_field37    0
form_field38    0
form_field39    0
form_field40    0
form_field41    0
form_field42    0
form_field43    0
form_field44    0
form_field45    0
form_field46    0
form_field47    0
form_field48    0
form_field49    0
form_field50    0
dtype: int64

In [6]:
#Encoding the Categorical Features

from sklearn.preprocessing import LabelEncoder

train["form_field47"] = LabelEncoder().fit_transform(train["form_field47"])
train["default_status"] = LabelEncoder().fit_transform(train["default_status"])
test["form_field47"] = LabelEncoder().fit_transform(test["form_field47"])

train.head()


,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,0,-99999.000000,1.129518,0.044335,0
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,-99999.0,0.0,0,349.805730,1.620483,0.322436,0
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,-99999.0,956940.0,-99999.0,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,0,-99999.000000,1.513370,0.011640,1
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,-99999.0,0.0,0,89.940100,0.664452,0.082729,0
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,0,97.887502,1.427891,0.045630,0


In [7]:
#Feature Engineering

In [8]:
#Define your X and y

X = train.drop(["Applicant_ID", "default_status"], axis =1)
y = train["default_status"]

X_testing = test.drop(["Applicant_ID"], axis = 1)
print("X_testing set shape: ", X_testing.shape)


X_testing set shape:  (24000, 50)


In [9]:
# #Split your data

# from sklearn.model_selection import train_test_split

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 0)
# ## "stratify = y", to make sure the districution of the target "CHURN" are evenly distributed in the train and validation sets


In [10]:
# #Standardize The Data

# # Import `StandardScaler` from `sklearn.preprocessing`
# from sklearn.preprocessing import StandardScaler

# cols = train.columns
# # Define the scaler 
# scaler = StandardScaler()

# # Scale the train data
# train = pd.DataFrame(scaler.fit_transform(train), columns = cols)

# # Scale the test data
# test = pd.DataFrame(scaler.transform(test), columns = cols)



In [11]:
#Building of Model

#Using lightgbm
import lightgbm as lgb
model = lgb.LGBMClassifier(max_depth=6, num_leaves=24, bagging_freq=7,feature_fraction=1.0, min_child_samples= 100,bagging_fraction=0.95, reg_alpha=3, n_estimators=5000, subsample=0.8, reg_lambda=2, random_state=50, eval_metric="auc")

 
                            
                            

In [12]:
from sklearn.model_selection import KFold,StratifiedKFold
skf = StratifiedKFold(n_splits=10,random_state=2020)

In [13]:

scores = []
pred = np.zeros(len(X_testing))
for (train_index,test_index) in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train,X_test = X.iloc[train_index], X.iloc[test_index]
    y_train,y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train,early_stopping_rounds=300,verbose=50, eval_set=[(X_test, y_test)])
    scores.append(roc_auc_score(y_test,model.predict_proba(X_test)[:,1]))
    pred += model.predict_proba(X_testing,num_iteration = model.best_iteration_)[:,1]




[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.95, subsample=0.8 will be ignored. Current value: bagging_fraction=0.95
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
Training until validation scores don't improve for 300 rounds
[50]	valid_0's binary_logloss: 0.418674
[100]	valid_0's binary_logloss: 0.414709
[150]	valid_0's binary_logloss: 0.413415
[200]	valid_0's binary_logloss: 0.413932
[250]	valid_0's binary_logloss: 0.413327
[300]	valid_0's binary_logloss: 0.413609
[350]	valid_0's binary_logloss: 0.414859
[400]	valid_0's binary_logloss: 0.415589
[450]	valid_0's binary_logloss: 0.416799
[500]	valid_0's binary_logloss: 0.417638
Early stopping, best iteration is:
[246]	valid_0's binary_logloss: 0.413182


LGBMClassifier(bagging_fraction=0.95, bagging_freq=7, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, eval_metric='auc',
               feature_fraction=1.0, importance_type='split', learning_rate=0.1,
               max_depth=6, min_child_samples=100, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=5000, n_jobs=-1, num_leaves=24,
               objective=None, random_state=50, reg_alpha=3, reg_lambda=2,
               silent=True, subsample=0.8, subsample_for_bin=200000,
               subsample_freq=0)

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.95, subsample=0.8 will be ignored. Current value: bagging_fraction=0.95
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
Training until validation scores don't improve for 300 rounds
[50]	valid_0's binary_logloss: 0.404155
[100]	valid_0's binary_logloss: 0.401132
[150]	valid_0's binary_logloss: 0.401062
[200]	valid_0's binary_logloss: 0.401499
[250]	valid_0's binary_logloss: 0.40227
[300]	valid_0's binary_logloss: 0.40345
[350]	valid_0's binary_logloss: 0.405044
[400]	valid_0's binary_logloss: 0.405895
Early stopping, best iteration is:
[141]	valid_0's binary_logloss: 0.40081


LGBMClassifier(bagging_fraction=0.95, bagging_freq=7, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, eval_metric='auc',
               feature_fraction=1.0, importance_type='split', learning_rate=0.1,
               max_depth=6, min_child_samples=100, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=5000, n_jobs=-1, num_leaves=24,
               objective=None, random_state=50, reg_alpha=3, reg_lambda=2,
               silent=True, subsample=0.8, subsample_for_bin=200000,
               subsample_freq=0)

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.95, subsample=0.8 will be ignored. Current value: bagging_fraction=0.95
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
Training until validation scores don't improve for 300 rounds
[50]	valid_0's binary_logloss: 0.41063
[100]	valid_0's binary_logloss: 0.406988
[150]	valid_0's binary_logloss: 0.40626
[200]	valid_0's binary_logloss: 0.407881
[250]	valid_0's binary_logloss: 0.409062
[300]	valid_0's binary_logloss: 0.409112
[350]	valid_0's binary_logloss: 0.410086
[400]	valid_0's binary_logloss: 0.412087
Early stopping, best iteration is:
[149]	valid_0's binary_logloss: 0.406171


...

LGBMClassifier(bagging_fraction=0.95, bagging_freq=7, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, eval_metric='auc',
               feature_fraction=1.0, importance_type='split', learning_rate=0.1,
               max_depth=6, min_child_samples=100, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=5000, n_jobs=-1, num_leaves=24,
               objective=None, random_state=50, reg_alpha=3, reg_lambda=2,
               silent=True, subsample=0.8, subsample_for_bin=200000,
               subsample_freq=0)

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.95, subsample=0.8 will be ignored. Current value: bagging_fraction=0.95
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
Training until validation scores don't improve for 300 rounds
[50]	valid_0's binary_logloss: 0.399938
[100]	valid_0's binary_logloss: 0.397821
[150]	valid_0's binary_logloss: 0.398831
[200]	valid_0's binary_logloss: 0.399694
[250]	valid_0's binary_logloss: 0.400385
[300]	valid_0's binary_logloss: 0.401223
[350]	valid_0's binary_logloss: 0.402471
Early stopping, best iteration is:
[94]	valid_0's binary_logloss: 0.397528


LGBMClassifier(bagging_fraction=0.95, bagging_freq=7, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, eval_metric='auc',
               feature_fraction=1.0, importance_type='split', learning_rate=0.1,
               max_depth=6, min_child_samples=100, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=5000, n_jobs=-1, num_leaves=24,
               objective=None, random_state=50, reg_alpha=3, reg_lambda=2,
               silent=True, subsample=0.8, subsample_for_bin=200000,
               subsample_freq=0)

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.95, subsample=0.8 will be ignored. Current value: bagging_fraction=0.95
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
Training until validation scores don't improve for 300 rounds
[50]	valid_0's binary_logloss: 0.406147
[100]	valid_0's binary_logloss: 0.401048
[150]	valid_0's binary_logloss: 0.399854
[200]	valid_0's binary_logloss: 0.401073
[250]	valid_0's binary_logloss: 0.401981
[300]	valid_0's binary_logloss: 0.402189
[350]	valid_0's binary_logloss: 0.402406
[400]	valid_0's binary_logloss: 0.403303
[450]	valid_0's binary_logloss: 0.404093
Early stopping, best iteration is:
[153]	valid_0's binary_logloss: 0.399716


LGBMClassifier(bagging_fraction=0.95, bagging_freq=7, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, eval_metric='auc',
               feature_fraction=1.0, importance_type='split', learning_rate=0.1,
               max_depth=6, min_child_samples=100, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=5000, n_jobs=-1, num_leaves=24,
               objective=None, random_state=50, reg_alpha=3, reg_lambda=2,
               silent=True, subsample=0.8, subsample_for_bin=200000,
               subsample_freq=0)

In [14]:
#Using CatBoost
from sklearn.metrics import log_loss, confusion_matrix, accuracy_score
model1 = CatBoostClassifier(random_seed = 14, n_estimators = 5000, max_depth=10, use_best_model=True, eval_metric="AUC")


In [15]:

scores1 = []
pred1 = np.zeros(len(X_testing))
for (train_index,test_index) in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train,X_test = X.iloc[train_index], X.iloc[test_index]
    y_train,y_test = y.iloc[train_index], y.iloc[test_index]
    model1.fit(X_train, y_train,early_stopping_rounds=300,eval_set=[(X_test, y_test)])
    scores1.append(roc_auc_score(y_test,model1.predict_proba(X_test)[:,1]))
    pred1 += model1.predict_proba(X_testing)[:,1]


#to be very sure of the X_testing shape
print("X_testing set shape: ", X_testing.shape)




Learning rate set to 0.041434
0:	test: 0.7962252	best: 0.7962252 (0)	total: 511ms	remaining: 42m 33s
1:	test: 0.8101925	best: 0.8101925 (1)	total: 801ms	remaining: 33m 22s
2:	test: 0.8120765	best: 0.8120765 (2)	total: 1.11s	remaining: 30m 58s
3:	test: 0.8145368	best: 0.8145368 (3)	total: 1.42s	remaining: 29m 28s
4:	test: 0.8153127	best: 0.8153127 (4)	total: 1.69s	remaining: 28m 5s
5:	test: 0.8166654	best: 0.8166654 (5)	total: 1.96s	remaining: 27m 14s
6:	test: 0.8181689	best: 0.8181689 (6)	total: 2.27s	remaining: 26m 57s
7:	test: 0.8173994	best: 0.8181689 (6)	total: 2.57s	remaining: 26m 42s
8:	test: 0.8180091	best: 0.8181689 (6)	total: 2.85s	remaining: 26m 18s
9:	test: 0.8187304	best: 0.8187304 (9)	total: 3.17s	remaining: 26m 20s
10:	test: 0.8192272	best: 0.8192272 (10)	total: 3.48s	remaining: 26m 18s
11:	test: 0.8194508	best: 0.8194508 (11)	total: 3.8s	remaining: 26m 20s
12:	test: 0.8197732	best: 0.8197732 (12)	total: 4.1s	remaining: 26m 13s
13:	test: 0.8201406	best: 0.8201406 (13)	tot

113:	test: 0.8280417	best: 0.8280626 (109)	total: 37.7s	remaining: 26m 54s
114:	test: 0.8280769	best: 0.8280769 (114)	total: 38s	remaining: 26m 52s
115:	test: 0.8280748	best: 0.8280769 (114)	total: 38.3s	remaining: 26m 51s
116:	test: 0.8280136	best: 0.8280769 (114)	total: 38.5s	remaining: 26m 48s
117:	test: 0.8280015	best: 0.8280769 (114)	total: 38.8s	remaining: 26m 46s
118:	test: 0.8280654	best: 0.8280769 (114)	total: 39.2s	remaining: 26m 47s
119:	test: 0.8280210	best: 0.8280769 (114)	total: 39.5s	remaining: 26m 46s
120:	test: 0.8280726	best: 0.8280769 (114)	total: 39.8s	remaining: 26m 45s
121:	test: 0.8281928	best: 0.8281928 (121)	total: 40.2s	remaining: 26m 46s
122:	test: 0.8282466	best: 0.8282466 (122)	total: 40.5s	remaining: 26m 44s
123:	test: 0.8283465	best: 0.8283465 (123)	total: 40.7s	remaining: 26m 41s
124:	test: 0.8284472	best: 0.8284472 (124)	total: 41s	remaining: 26m 38s
125:	test: 0.8284790	best: 0.8284790 (125)	total: 41.3s	remaining: 26m 36s
126:	test: 0.8284972	best: 0.

224:	test: 0.8305607	best: 0.8307104 (221)	total: 1m 11s	remaining: 25m 17s
225:	test: 0.8305697	best: 0.8307104 (221)	total: 1m 11s	remaining: 25m 15s
226:	test: 0.8306485	best: 0.8307104 (221)	total: 1m 12s	remaining: 25m 14s
227:	test: 0.8306439	best: 0.8307104 (221)	total: 1m 12s	remaining: 25m 13s
228:	test: 0.8305771	best: 0.8307104 (221)	total: 1m 12s	remaining: 25m 12s
229:	test: 0.8306230	best: 0.8307104 (221)	total: 1m 12s	remaining: 25m 10s
230:	test: 0.8306449	best: 0.8307104 (221)	total: 1m 13s	remaining: 25m 9s
231:	test: 0.8306502	best: 0.8307104 (221)	total: 1m 13s	remaining: 25m 8s
232:	test: 0.8306425	best: 0.8307104 (221)	total: 1m 13s	remaining: 25m 6s
233:	test: 0.8305887	best: 0.8307104 (221)	total: 1m 13s	remaining: 25m 5s
234:	test: 0.8305994	best: 0.8307104 (221)	total: 1m 14s	remaining: 25m 4s
235:	test: 0.8306413	best: 0.8307104 (221)	total: 1m 14s	remaining: 25m 2s
236:	test: 0.8306080	best: 0.8307104 (221)	total: 1m 14s	remaining: 25m 1s
237:	test: 0.830552

333:	test: 0.8320462	best: 0.8321025 (321)	total: 1m 48s	remaining: 25m 15s
334:	test: 0.8320330	best: 0.8321025 (321)	total: 1m 48s	remaining: 25m 17s
335:	test: 0.8321164	best: 0.8321164 (335)	total: 1m 49s	remaining: 25m 19s
336:	test: 0.8321132	best: 0.8321164 (335)	total: 1m 49s	remaining: 25m 21s
337:	test: 0.8320543	best: 0.8321164 (335)	total: 1m 50s	remaining: 25m 24s
338:	test: 0.8320618	best: 0.8321164 (335)	total: 1m 51s	remaining: 25m 26s
339:	test: 0.8321164	best: 0.8321164 (335)	total: 1m 51s	remaining: 25m 29s
340:	test: 0.8321273	best: 0.8321273 (340)	total: 1m 51s	remaining: 25m 29s
341:	test: 0.8320673	best: 0.8321273 (340)	total: 1m 52s	remaining: 25m 29s
342:	test: 0.8320642	best: 0.8321273 (340)	total: 1m 52s	remaining: 25m 28s
343:	test: 0.8320871	best: 0.8321273 (340)	total: 1m 52s	remaining: 25m 28s
344:	test: 0.8320255	best: 0.8321273 (340)	total: 1m 53s	remaining: 25m 27s
345:	test: 0.8320487	best: 0.8321273 (340)	total: 1m 53s	remaining: 25m 26s
346:	test: 0

442:	test: 0.8316408	best: 0.8321273 (340)	total: 2m 20s	remaining: 24m 4s
443:	test: 0.8316304	best: 0.8321273 (340)	total: 2m 20s	remaining: 24m 3s
444:	test: 0.8317111	best: 0.8321273 (340)	total: 2m 20s	remaining: 24m 2s
445:	test: 0.8316649	best: 0.8321273 (340)	total: 2m 21s	remaining: 24m 1s
446:	test: 0.8317077	best: 0.8321273 (340)	total: 2m 21s	remaining: 24m 1s
447:	test: 0.8317223	best: 0.8321273 (340)	total: 2m 21s	remaining: 24m
448:	test: 0.8316727	best: 0.8321273 (340)	total: 2m 22s	remaining: 24m
449:	test: 0.8317285	best: 0.8321273 (340)	total: 2m 22s	remaining: 23m 59s
450:	test: 0.8318369	best: 0.8321273 (340)	total: 2m 22s	remaining: 23m 59s
451:	test: 0.8318527	best: 0.8321273 (340)	total: 2m 23s	remaining: 23m 59s
452:	test: 0.8318065	best: 0.8321273 (340)	total: 2m 23s	remaining: 23m 58s
453:	test: 0.8317811	best: 0.8321273 (340)	total: 2m 23s	remaining: 23m 58s
454:	test: 0.8317287	best: 0.8321273 (340)	total: 2m 23s	remaining: 23m 57s
455:	test: 0.8316102	best

...

332:	test: 0.8459124	best: 0.8459448 (331)	total: 1m 34s	remaining: 22m 6s
333:	test: 0.8458838	best: 0.8459448 (331)	total: 1m 34s	remaining: 22m 6s
334:	test: 0.8458546	best: 0.8459448 (331)	total: 1m 35s	remaining: 22m 5s
335:	test: 0.8458686	best: 0.8459448 (331)	total: 1m 35s	remaining: 22m 5s
336:	test: 0.8459189	best: 0.8459448 (331)	total: 1m 35s	remaining: 22m 4s
337:	test: 0.8459358	best: 0.8459448 (331)	total: 1m 35s	remaining: 22m 3s
338:	test: 0.8459287	best: 0.8459448 (331)	total: 1m 36s	remaining: 22m 3s
339:	test: 0.8459318	best: 0.8459448 (331)	total: 1m 36s	remaining: 22m 2s
340:	test: 0.8458936	best: 0.8459448 (331)	total: 1m 36s	remaining: 22m 2s
341:	test: 0.8458843	best: 0.8459448 (331)	total: 1m 37s	remaining: 22m 2s
342:	test: 0.8459268	best: 0.8459448 (331)	total: 1m 37s	remaining: 22m 1s
343:	test: 0.8458919	best: 0.8459448 (331)	total: 1m 37s	remaining: 22m
344:	test: 0.8458750	best: 0.8459448 (331)	total: 1m 37s	remaining: 22m
345:	test: 0.8458856	best: 0.84

441:	test: 0.8462108	best: 0.8462418 (436)	total: 2m 5s	remaining: 21m 35s
442:	test: 0.8462094	best: 0.8462418 (436)	total: 2m 5s	remaining: 21m 35s
443:	test: 0.8462094	best: 0.8462418 (436)	total: 2m 6s	remaining: 21m 34s
444:	test: 0.8461851	best: 0.8462418 (436)	total: 2m 6s	remaining: 21m 34s
445:	test: 0.8461416	best: 0.8462418 (436)	total: 2m 6s	remaining: 21m 34s
446:	test: 0.8461987	best: 0.8462418 (436)	total: 2m 6s	remaining: 21m 33s
447:	test: 0.8463016	best: 0.8463016 (447)	total: 2m 7s	remaining: 21m 33s
448:	test: 0.8463108	best: 0.8463108 (448)	total: 2m 7s	remaining: 21m 32s
449:	test: 0.8463185	best: 0.8463185 (449)	total: 2m 7s	remaining: 21m 32s
450:	test: 0.8463740	best: 0.8463740 (450)	total: 2m 8s	remaining: 21m 31s
451:	test: 0.8463676	best: 0.8463740 (450)	total: 2m 8s	remaining: 21m 31s
452:	test: 0.8463364	best: 0.8463740 (450)	total: 2m 8s	remaining: 21m 30s
453:	test: 0.8463659	best: 0.8463740 (450)	total: 2m 8s	remaining: 21m 30s
454:	test: 0.8463208	best

550:	test: 0.8459053	best: 0.8464180 (465)	total: 2m 36s	remaining: 21m 4s
551:	test: 0.8459112	best: 0.8464180 (465)	total: 2m 36s	remaining: 21m 4s
552:	test: 0.8458841	best: 0.8464180 (465)	total: 2m 37s	remaining: 21m 4s
553:	test: 0.8458784	best: 0.8464180 (465)	total: 2m 37s	remaining: 21m 4s
554:	test: 0.8458975	best: 0.8464180 (465)	total: 2m 37s	remaining: 21m 4s
555:	test: 0.8458929	best: 0.8464180 (465)	total: 2m 38s	remaining: 21m 3s
556:	test: 0.8458620	best: 0.8464180 (465)	total: 2m 38s	remaining: 21m 3s
557:	test: 0.8459131	best: 0.8464180 (465)	total: 2m 38s	remaining: 21m 3s
558:	test: 0.8459270	best: 0.8464180 (465)	total: 2m 38s	remaining: 21m 3s
559:	test: 0.8458675	best: 0.8464180 (465)	total: 2m 39s	remaining: 21m 2s
560:	test: 0.8458951	best: 0.8464180 (465)	total: 2m 39s	remaining: 21m 2s
561:	test: 0.8458843	best: 0.8464180 (465)	total: 2m 40s	remaining: 21m 3s
562:	test: 0.8459075	best: 0.8464180 (465)	total: 2m 40s	remaining: 21m 3s
563:	test: 0.8458803	best

660:	test: 0.8452307	best: 0.8464180 (465)	total: 3m 7s	remaining: 20m 33s
661:	test: 0.8452259	best: 0.8464180 (465)	total: 3m 8s	remaining: 20m 33s
662:	test: 0.8452704	best: 0.8464180 (465)	total: 3m 8s	remaining: 20m 33s
663:	test: 0.8452273	best: 0.8464180 (465)	total: 3m 8s	remaining: 20m 33s
664:	test: 0.8451945	best: 0.8464180 (465)	total: 3m 9s	remaining: 20m 32s
665:	test: 0.8451602	best: 0.8464180 (465)	total: 3m 9s	remaining: 20m 32s
666:	test: 0.8451023	best: 0.8464180 (465)	total: 3m 9s	remaining: 20m 31s
667:	test: 0.8450232	best: 0.8464180 (465)	total: 3m 9s	remaining: 20m 31s
668:	test: 0.8450171	best: 0.8464180 (465)	total: 3m 10s	remaining: 20m 30s
669:	test: 0.8450182	best: 0.8464180 (465)	total: 3m 10s	remaining: 20m 30s
670:	test: 0.8450263	best: 0.8464180 (465)	total: 3m 10s	remaining: 20m 29s
671:	test: 0.8450440	best: 0.8464180 (465)	total: 3m 10s	remaining: 20m 29s
672:	test: 0.8449916	best: 0.8464180 (465)	total: 3m 11s	remaining: 20m 28s
673:	test: 0.8449716

X_testing set shape:  (24000, 50)


In [16]:
#Using XGBoost
from sklearn.metrics import log_loss, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score
model2 = XGBClassifier(objective ='binary:logistic', random_seed=20920, n_estimators=5000, learning_rate=0.05, max_depth=15, reg_lambda=500, subsample=0.8, col_sample_bytree = 0.8, use_best_model=True, eval_metric="auc")


                         

In [19]:

scores2 = []
pred2 = np.zeros(len(X_testing))
for (train_index,test_index) in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train,X_test = X.iloc[train_index], X.iloc[test_index]
    y_train,y_test = y.iloc[train_index], y.iloc[test_index]
    model2.fit(X_train, y_train,early_stopping_rounds=300,eval_set=[(X_test, y_test)])
    scores2.append(roc_auc_score(y_test,model2.predict_proba(X_test)[:,1]))
    pred2 += model2.predict_proba(X_testing,ntree_limit = model2.get_booster().best_ntree_limit)[:,1]





[0]	validation_0-auc:0.731881
Will train until validation_0-auc hasn't improved in 300 rounds.
[1]	validation_0-auc:0.738683
[2]	validation_0-auc:0.760873
[3]	validation_0-auc:0.770718
[4]	validation_0-auc:0.77687
[5]	validation_0-auc:0.779988
[6]	validation_0-auc:0.78103
[7]	validation_0-auc:0.781917
[8]	validation_0-auc:0.786
[9]	validation_0-auc:0.787781
[10]	validation_0-auc:0.789191
[11]	validation_0-auc:0.79125
[12]	validation_0-auc:0.793576
[13]	validation_0-auc:0.7947
[14]	validation_0-auc:0.795559
[15]	validation_0-auc:0.796467
[16]	validation_0-auc:0.79729
[17]	validation_0-auc:0.797823
[18]	validation_0-auc:0.79922
[19]	validation_0-auc:0.799498
[20]	validation_0-auc:0.799975
[21]	validation_0-auc:0.800475
[22]	validation_0-auc:0.80097
[23]	validation_0-auc:0.801195
[24]	validation_0-auc:0.803685
[25]	validation_0-auc:0.804084
[26]	validation_0-auc:0.805369
[27]	validation_0-auc:0.805871
[28]	validation_0-auc:0.807524
[29]	validation_0-auc:0.807936
[30]	validation_0-auc:0.80

[259]	validation_0-auc:0.83172
[260]	validation_0-auc:0.831723
[261]	validation_0-auc:0.831767
[262]	validation_0-auc:0.831811
[263]	validation_0-auc:0.831827
[264]	validation_0-auc:0.831845
[265]	validation_0-auc:0.831858
[266]	validation_0-auc:0.831864
[267]	validation_0-auc:0.831923
[268]	validation_0-auc:0.831922
[269]	validation_0-auc:0.831929
[270]	validation_0-auc:0.831972
[271]	validation_0-auc:0.831969
[272]	validation_0-auc:0.83197
[273]	validation_0-auc:0.832008
[274]	validation_0-auc:0.831998
[275]	validation_0-auc:0.832051
[276]	validation_0-auc:0.832038
[277]	validation_0-auc:0.832016
[278]	validation_0-auc:0.831997
[279]	validation_0-auc:0.831998
[280]	validation_0-auc:0.832028
[281]	validation_0-auc:0.832011
[282]	validation_0-auc:0.83199
[283]	validation_0-auc:0.831977
[284]	validation_0-auc:0.831993
[285]	validation_0-auc:0.832027
[286]	validation_0-auc:0.832004
[287]	validation_0-auc:0.83202
[288]	validation_0-auc:0.832087
[289]	validation_0-auc:0.832106
[290]	valida

[517]	validation_0-auc:0.833555
[518]	validation_0-auc:0.833572
[519]	validation_0-auc:0.833554
[520]	validation_0-auc:0.833568
[521]	validation_0-auc:0.83356
[522]	validation_0-auc:0.833592
[523]	validation_0-auc:0.833611
[524]	validation_0-auc:0.833594
[525]	validation_0-auc:0.833576
[526]	validation_0-auc:0.833586
[527]	validation_0-auc:0.833533
[528]	validation_0-auc:0.833559
[529]	validation_0-auc:0.833559
[530]	validation_0-auc:0.833598
[531]	validation_0-auc:0.833595
[532]	validation_0-auc:0.833614
[533]	validation_0-auc:0.833615
[534]	validation_0-auc:0.8336
[535]	validation_0-auc:0.83364
[536]	validation_0-auc:0.833644
[537]	validation_0-auc:0.833647
[538]	validation_0-auc:0.833647
[539]	validation_0-auc:0.833672
[540]	validation_0-auc:0.83367
[541]	validation_0-auc:0.833676
[542]	validation_0-auc:0.83368
[543]	validation_0-auc:0.833678
[544]	validation_0-auc:0.833727
[545]	validation_0-auc:0.833673
[546]	validation_0-auc:0.833666
[547]	validation_0-auc:0.833659
[548]	validati

[774]	validation_0-auc:0.832954
[775]	validation_0-auc:0.832944
[776]	validation_0-auc:0.832955
[777]	validation_0-auc:0.832969
[778]	validation_0-auc:0.832941
[779]	validation_0-auc:0.832932
[780]	validation_0-auc:0.832933
[781]	validation_0-auc:0.832917
[782]	validation_0-auc:0.832907
[783]	validation_0-auc:0.832909
[784]	validation_0-auc:0.832861
[785]	validation_0-auc:0.832879
[786]	validation_0-auc:0.832918
[787]	validation_0-auc:0.832895
[788]	validation_0-auc:0.832879
[789]	validation_0-auc:0.832899
[790]	validation_0-auc:0.832912
[791]	validation_0-auc:0.832903
[792]	validation_0-auc:0.832921
[793]	validation_0-auc:0.832941
[794]	validation_0-auc:0.832874
[795]	validation_0-auc:0.832894
[796]	validation_0-auc:0.832889
[797]	validation_0-auc:0.832867
[798]	validation_0-auc:0.832887
[799]	validation_0-auc:0.832878
[800]	validation_0-auc:0.832899
[801]	validation_0-auc:0.832902
[802]	validation_0-auc:0.832905
[803]	validation_0-auc:0.83292
[804]	validation_0-auc:0.832938
[805]	val

XGBClassifier(base_score=0.5, booster='gbtree', col_sample_bytree=0.8,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              eval_metric='auc', gamma=0, learning_rate=0.05, max_delta_step=0,
              max_depth=15, min_child_weight=1, missing=None, n_estimators=5000,
              n_jobs=1, nthread=None, objective='binary:logistic',
              random_seed=20920, random_state=0, reg_alpha=0, reg_lambda=500,
              scale_pos_weight=1, seed=None, silent=None, subsample=0.8,
              use_best_model=True, verbosity=1)

...

[0]	validation_0-auc:0.75314
Will train until validation_0-auc hasn't improved in 300 rounds.
[1]	validation_0-auc:0.769566
[2]	validation_0-auc:0.77769
[3]	validation_0-auc:0.792387
[4]	validation_0-auc:0.797534
[5]	validation_0-auc:0.803742
[6]	validation_0-auc:0.805006
[7]	validation_0-auc:0.806988
[8]	validation_0-auc:0.807113
[9]	validation_0-auc:0.808839
[10]	validation_0-auc:0.810809
[11]	validation_0-auc:0.811536
[12]	validation_0-auc:0.813614
[13]	validation_0-auc:0.814141
[14]	validation_0-auc:0.814645
[15]	validation_0-auc:0.816342
[16]	validation_0-auc:0.817778
[17]	validation_0-auc:0.818602
[18]	validation_0-auc:0.819664
[19]	validation_0-auc:0.820366
[20]	validation_0-auc:0.821263
[21]	validation_0-auc:0.822131
[22]	validation_0-auc:0.822738
[23]	validation_0-auc:0.823124
[24]	validation_0-auc:0.823538
[25]	validation_0-auc:0.824425
[26]	validation_0-auc:0.825274
[27]	validation_0-auc:0.825483
[28]	validation_0-auc:0.826125
[29]	validation_0-auc:0.826567
[30]	validation_0

[259]	validation_0-auc:0.84604
[260]	validation_0-auc:0.846084
[261]	validation_0-auc:0.846144
[262]	validation_0-auc:0.846134
[263]	validation_0-auc:0.84619
[264]	validation_0-auc:0.846205
[265]	validation_0-auc:0.846289
[266]	validation_0-auc:0.846324
[267]	validation_0-auc:0.846351
[268]	validation_0-auc:0.846305
[269]	validation_0-auc:0.846345
[270]	validation_0-auc:0.846356
[271]	validation_0-auc:0.846363
[272]	validation_0-auc:0.846363
[273]	validation_0-auc:0.846426
[274]	validation_0-auc:0.846481
[275]	validation_0-auc:0.846514
[276]	validation_0-auc:0.846521
[277]	validation_0-auc:0.846541
[278]	validation_0-auc:0.846556
[279]	validation_0-auc:0.846562
[280]	validation_0-auc:0.846563
[281]	validation_0-auc:0.846565
[282]	validation_0-auc:0.846595
[283]	validation_0-auc:0.846598
[284]	validation_0-auc:0.846638
[285]	validation_0-auc:0.846648
[286]	validation_0-auc:0.846694
[287]	validation_0-auc:0.846702
[288]	validation_0-auc:0.846716
[289]	validation_0-auc:0.846753
[290]	vali

[516]	validation_0-auc:0.847361
[517]	validation_0-auc:0.847345
[518]	validation_0-auc:0.847353
[519]	validation_0-auc:0.847345
[520]	validation_0-auc:0.847399
[521]	validation_0-auc:0.847395
[522]	validation_0-auc:0.847371
[523]	validation_0-auc:0.847382
[524]	validation_0-auc:0.847386
[525]	validation_0-auc:0.847353
[526]	validation_0-auc:0.847332
[527]	validation_0-auc:0.847344
[528]	validation_0-auc:0.847378
[529]	validation_0-auc:0.847366
[530]	validation_0-auc:0.847365
[531]	validation_0-auc:0.847356
[532]	validation_0-auc:0.84736
[533]	validation_0-auc:0.847348
[534]	validation_0-auc:0.847345
[535]	validation_0-auc:0.847347
[536]	validation_0-auc:0.847346
[537]	validation_0-auc:0.847361
[538]	validation_0-auc:0.847364
[539]	validation_0-auc:0.847392
[540]	validation_0-auc:0.847403
[541]	validation_0-auc:0.847357
[542]	validation_0-auc:0.847357
[543]	validation_0-auc:0.847342
[544]	validation_0-auc:0.84732
[545]	validation_0-auc:0.847331
[546]	validation_0-auc:0.847345
[547]	vali

[773]	validation_0-auc:0.846855
[774]	validation_0-auc:0.846852
[775]	validation_0-auc:0.846859
[776]	validation_0-auc:0.846852
[777]	validation_0-auc:0.846823
[778]	validation_0-auc:0.84683
[779]	validation_0-auc:0.846834
[780]	validation_0-auc:0.846824
[781]	validation_0-auc:0.846832
Stopping. Best iteration:
[481]	validation_0-auc:0.847593



XGBClassifier(base_score=0.5, booster='gbtree', col_sample_bytree=0.8,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              eval_metric='auc', gamma=0, learning_rate=0.05, max_delta_step=0,
              max_depth=15, min_child_weight=1, missing=None, n_estimators=5000,
              n_jobs=1, nthread=None, objective='binary:logistic',
              random_seed=20920, random_state=0, reg_alpha=0, reg_lambda=500,
              scale_pos_weight=1, seed=None, silent=None, subsample=0.8,
              use_best_model=True, verbosity=1)

In [20]:
print("scores",np.mean(scores))
print("scores1",np.mean(scores1))
print("scores2",np.mean(scores2))

scores 0.8394756078629401
scores1 0.8404470171633435
scores2 0.8398292210915388


In [21]:
pred_av = pred/10
pred_av1 = pred1/10
pred_av2 = pred2/10

In [22]:
total_av = (pred_av + pred_av1 + pred_av2)/3

In [23]:
sub = sample.copy()
sub['default_status'] = total_av
sub.to_csv('LGB_XGB_CAT.csv',index=False)
sub.describe()

,default_status
count,24000.000000
mean,0.243193
std,0.232531
min,0.003238
25%,0.045703
50%,0.166789
75%,0.383515
max,0.943318


In [24]:
blend = ((pred_av*0.4) + (pred_av1*0.6))*((pred_av2*0.4) + (pred_av1*0.6)) 
blend_sub = sample.copy()
blend_sub['default_status'] = blend
blend_sub.to_csv('LGB_XGB_CAT6.csv',index=False)
blend_sub.describe()

,default_status
count,24000.000000
mean,0.112955
std,0.174542
min,0.000006
25%,0.002079
50%,0.027766
75%,0.145928
max,0.909245


In [25]:
sample = pd.read_csv('SampleSubmission.csv')
a1 = pd.read_csv('LGB_XGB_CAT3.csv')
a2 = pd.read_csv('LGB_XGB_CAT6.csv')
a3 = (a1.default_status + a2.default_status)/2
a3_sub = sample.copy()
a3_sub.default_status = a3
a3_sub.to_csv('LGB_XGB_CAT22.csv', index = False)
a3.describe()

count    24000.000000
mean         0.178074
std          0.200964
min          0.001622
25%          0.023909
50%          0.097172
75%          0.264639
max          0.926281
Name: default_status, dtype: float64